In [ ]:
from difflib import SequenceMatcher
import re

def contains_only_symbols(input_string):
    # Define a regular expression pattern to match symbols
    symbol_pattern = r'^[\W_]+$'

    # Use re.match to check if the entire string matches the pattern
    if re.match(symbol_pattern, input_string):
        return True
    elif input_string == '':
        return True
    else:
        return False

def longestSubstring(str1,str2):
    seqMatch = SequenceMatcher(None,str1,str2)
    match = seqMatch.find_longest_match(0, len(str1), 0, len(str2))
    if (match.size!=0):
        return str1[match.a: match.a + match.size]
    else:
        return ''

def find_longest_common_subsequence(str1, str_list):
    indices = []
    subs = []
    loop = 0
    while not contains_only_symbols(str1):
      if loop > 20:
        break
      lcs = [""]*len(str_list)
      for i, s in enumerate(str_list):
          lcs[i] = longestSubstring(str1, s)
      idx = lcs.index(max(lcs, key=len))
      if not contains_only_symbols(lcs[idx]):  
        indices.append(idx)
        subs.append(lcs[idx])
      str1 = str1.replace(lcs[idx], '')
      lcs[idx] = ''
      loop += 1
    return indices, subs

## ChatGPT-EXT

In [ ]:
import openai
import pandas as pd
import os

m = 6

openai.api_key = "Your API Key"

# Get the list of files in the directory
directory = 'input_docs/docs'
file_list = os.listdir(directory)

dir_dict = {'White': 'H-A', 'Hisp': 'A-W', 'AA': 'W-H'}

for file in file_list[24:]:
    df = pd.DataFrame()
    docs = pd.read_csv('input_docs/docs/' + file)
    dict = {}

    for remove_label in ['White', 'Hisp', 'AA']:
        docs_filter = docs[docs.label != remove_label]
        document = ''
        for idx, sentence in enumerate(docs_filter.text, start=1):
            document += f"\n{idx}. {sentence}"
        for i in range(10):
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are an extractive summarizer that follows the output pattern."},
                    {"role": "user", "content": f"Please extract sentences as the summary. The summary should contain {m} sentences. Document: {document}"}
                ]
            )
            summary = response.choices[0].message.content
            indices, subs = find_longest_common_subsequence(summary, docs['text'].values)

            persentage = []
            for i in range(len(indices)):
                persentage.append(len(subs[i])/len(docs['text'][indices[i]]))
            if sum(sorted(persentage, reverse=True)[:6]) > 3:
                top_indices = sorted(range(len(persentage)), key=lambda i: persentage[i], reverse=True)[:6]
                selected_values = [indices[i] for i in top_indices]
                docs.iloc[selected_values].to_csv(f'output_chatGPT/{dir_dict[remove_label]}/{file}', index = False)
                break

        dict[dir_dict[remove_label]] = response
    df = pd.concat([df, pd.DataFrame(dict)])
    df.to_csv(f'output_chatGPT/model_outputs/{file}', index=True)        

## FairGPT

In [ ]:
import openai
import pandas as pd
import os

m = 6

openai.api_key = "Your API Key"

# Get the list of files in the directory
directory = 'input_docs/docs'
file_list = os.listdir(directory)

dir_dict = {'White': 'H-A', 'Hisp': 'A-W', 'AA': 'W-H'}
name_dict = {'White': 'White', 'Hisp': 'Hispanic', 'AA': 'African-American'}


for file in file_list:
    docs = pd.read_csv(directory + '/' + file)
    dict = {}

    for remove_label in ['White', 'Hisp', 'AA']:
        print(remove_label)

        labels = list({'White', 'Hisp', 'AA'} - {remove_label})

        docs_filter = docs[docs.label == labels[0]]
        document_g1 = f'{name_dict[labels[0]]} People: \n'
        for idx, sentence in enumerate(docs_filter.text, start=1):
            document_g1 += f"**{{\n{idx}. {sentence}}}**"

        docs_filter = docs[docs.label == labels[1]]
        document_g2 = f'{name_dict[labels[1]]} People: \n'
        for idx, sentence in enumerate(docs_filter.text, start=1):
            document_g2 += f"**{{\n{idx}. {sentence}}}**"

        document = document_g1 + '\n\n' + document_g2

        for i in range(10):
            response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": "You are an extractive fair summarizer that follows the output pattern. A fair summarizer should select the same number of sentences from each group of people."},
                        {"role": "user", "content": f"Please extract sentences as the summary. The summary should contain {m} sentences which means select {m/2} number of sentences from each group of people to represent the idea of all groups in a fair manner. Document: \n\n {document}"}
                    ]
                )
            summary = response.choices[0].message.content
            indices, subs = find_longest_common_subsequence(summary, docs['text'].values)

            persentage = []
            for j in range(len(indices)):
                persentage.append(len(subs[j])/len(docs['text'][indices[j]]))
            print(i)
            if sum(sorted(persentage, reverse=True)[:6]) > 3:
                top_indices = sorted(range(len(persentage)), key=lambda i: persentage[i], reverse=True)[:6]
                selected_values = [indices[k] for k in top_indices]
                print(i)
                if (docs.iloc[selected_values]['label'].value_counts(normalize=True) == 0.5).all():
                    docs.iloc[selected_values].to_csv(f'output_fairChatGPT/{dir_dict[remove_label]}/{file}', index = False)
                    print(i)
                    break
                else:
                    print(docs.iloc[selected_values]['label'].value_counts(normalize=True))